In [ ]:
from oem2orm import oep_oedialect_oem2orm as oem2orm
import os
import pandas as pd
import getpass

In [ ]:
oem2orm.setup_logger()

In [ ]:
os.environ["OEP_TOKEN"] = getpass.getpass('Token:')

In [ ]:
# provide your OEP-username to create a connection to the database, 
# your token is taken from the environment variable you've created above
db = oem2orm.setup_db_connection()

In [ ]:
metadata_folder = oem2orm.select_oem_dir(oem_folder_name="metadata")

In [ ]:
ordered_orm = oem2orm.collect_ordered_tables_from_oem(db, metadata_folder)

In [ ]:
#create table
oem2orm.create_tables(db, ordered_orm)

## Writing data into a table

In [ ]:
example_df = pd.read_csv('data/upload_tutorial_example_data.csv', encoding='utf8', sep=';')

In [ ]:
table_name = "upload_tutorial_example_data3"
connection = db.engine.connect()
schema = "model_draft"

In [ ]:
try: 
    example_df.to_sql(table_name, connection, schema=schema, if_exists='replace', index=False)
    print('Inserted data to ' + schema + '.' + table_name)
except Exception as e:
    session.rollback()
    print('Writing to ' + table_name + ' failed!')

## Writing metadata to table

In [ ]:
import json
import requests
from oep_client import OepClient

In [ ]:
# The token needs to be passed to the oep-client 
cl = OepClient(token=os.environ["OEP_TOKEN"])

In [ ]:
# Read the metadata file into a json object
with open('metadata/example_metadata.json') as json_file:
    data = json.load(json_file)

There is a temporary bug in the metadata parsing tool of the OEP, that will be updated in a release of [omi](https://github.com/OpenEnergyPlatform/omi), which is soon to come. Until it's released, we can adapt our string to work around the bug and allow an upload to the OEP.

In [ ]:
# workaround metadata upload bug in omi-version of the OEP
data['temporal']
if "temporal" in data:
    if "timeseries" in data["temporal"]:
        if "alignment" in data["temporal"]["timeseries"]:
            data["temporal"]["timeseries"]["timestamp"] = data["temporal"]["timeseries"].pop("alignment")

In [ ]:
cl.update_metadata(metadata=data)

## Deleting your table

Running the following commands will delete the tables from the database which you have in your ordered ORM. Take care that you only delete tables you actually want to have removed.

In [ ]:
# You need to type yes if you want to actually delete the table
oem2orm.delete_tables(db, ordered_orm)